In [2]:
# read both datafiles
import pandas as pd
location_data = pd.read_csv("zip_codes_states.csv")
data = pd.read_csv("data_clean.csv")

In [257]:
cities = data["city_or_county"]
loc_cities = location_data["city"]
loc_counties = location_data["county"]
long = location_data["longitude"]
lat = location_data["latitude"]
date = data["date"]
longitude = data["longitude"]
latitude = data['latitude']

In [8]:
# make a list of all the missing longitude/latitude data of the clean data file
total_cities = []
for i in range(len(data)):
    if pd.isna(longitude[i]):
        total_cities.append(cities[i])

In [10]:
# make a list of all the (unknown) counties
list_counties = []
for city in set(total_cities):
    for county in loc_counties:
        if city == county and city not in list_counties:
            list_counties.append(county)

In [11]:
# make a list of all remainder cities/counties in total list
remainder = [x for x in set(total_cities) if x not in list_counties]

In [12]:
# make a list of all the cities in both databases
list_cities = []
for city in remainder:
    for city2 in loc_cities:
        if city == city2 and city not in list_cities:
            list_cities.append(city)

In [37]:
# make a dictionary of all the cities with their longitude/latitude
loc_dict = {}
for city in list_cities:
    for i in range(len(loc_cities)):
        if city == loc_cities[i] and city not in loc_dict:
            loc_dict[city] = [lat[i], long[i]]

In [39]:
# append the counties to the dictionary
for county in list_counties:
    for i in range(len(loc_counties)):
        if county == loc_counties[i] and county not in loc_dict:
            loc_dict[county] = [lat[i], long[i]]

In [187]:
# make a copy of the dictionary
import copy
temporary = copy.deepcopy(loc_dict)

In [75]:
n_killed = data["n_killed"]
n_injured = data["n_injured"]

killed_dict = {}

# make a dictionary with all the killed rate per city of the clean database
# do not append the 2018 data
for i in range(len(data)):
    if "2018" in date[i]:
        break
    if cities[i] not in killed_dict:
        killed_dict[cities[i]] = 0
    if cities[i] in killed_dict:
        killed_dict[cities[i]] += n_killed[i]
        
print(killed_dict)

{'Muskegon': 9, 'Newark': 310, 'Queens': 46, 'Brooklyn': 364, 'Springfield': 205, 'Columbus': 527, 'Syracuse': 80, 'Lexington': 99, 'Bibb County': 1, 'Prince William County': 0, 'Philadelphia': 854, 'Hampton': 73, 'North Charleston': 104, 'Baltimore': 1003, 'Wilmington': 177, 'Grand Junction': 13, 'Huntsville': 77, 'Camden': 134, 'New Orleans': 658, 'Hurricane': 2, 'Fort Wayne': 137, 'Marrero': 34, 'Pinckneyville': 1, 'Bronx': 215, 'Canon City': 8, 'Lubbock': 43, 'North Platte': 2, 'Johns Island': 2, 'Houston': 1063, 'Lynn': 15, 'Newport News': 102, 'Milton': 13, 'Fort Myers': 82, 'Tuscaloosa': 38, 'Bogue Chitto': 10, 'Clementon': 1, 'Olanta': 0, 'Janesville': 2, 'Crenshaw': 2, 'Tolleson': 10, 'Lihue': 6, 'Georgetown': 11, 'Homestead': 18, 'Orlando': 279, 'Cynthiana': 6, 'Toledo': 102, 'Wichita': 117, 'Clinton': 30, 'Washington': 441, 'Princeton': 15, 'Warner Robins': 18, 'Borger': 5, 'Milwaukee': 464, 'Manchester': 24, 'Deltona': 18, 'Providence': 38, 'Duplin County': 1, 'Rio Rico': 0

In [258]:
incidents_dict = {}

# make a dictionary of all the incidents per city
for i in range(len(data)):
    if "2018" in date[i]:
        break
    if cities[i] not in incidents_dict:
        incidents_dict[cities[i]] = 1
    if cities[i] in incidents_dict:
        incidents_dict[cities[i]] += 1
        
print(incidents_dict)

{'Muskegon': 63, 'Newark': 838, 'Queens': 131, 'Brooklyn': 1368, 'Springfield': 1679, 'Columbus': 2139, 'Syracuse': 634, 'Lexington': 906, 'Bibb County': 21, 'Prince William County': 4, 'Philadelphia': 2825, 'Hampton': 376, 'North Charleston': 375, 'Baltimore': 3704, 'Wilmington': 1243, 'Grand Junction': 155, 'Huntsville': 529, 'Camden': 282, 'New Orleans': 2927, 'Hurricane': 17, 'Fort Wayne': 573, 'Marrero': 119, 'Pinckneyville': 3, 'Bronx': 813, 'Canon City': 24, 'Lubbock': 298, 'North Platte': 13, 'Johns Island': 25, 'Houston': 2332, 'Lynn': 248, 'Newport News': 500, 'Milton': 62, 'Fort Myers': 368, 'Tuscaloosa': 205, 'Bogue Chitto': 4, 'Clementon': 6, 'Olanta': 3, 'Janesville': 35, 'Crenshaw': 3, 'Tolleson': 15, 'Lihue': 9, 'Georgetown': 79, 'Homestead': 42, 'Orlando': 966, 'Cynthiana': 10, 'Toledo': 777, 'Wichita': 747, 'Clinton': 108, 'Washington': 3096, 'Princeton': 43, 'Warner Robins': 78, 'Borger': 12, 'Milwaukee': 2295, 'Manchester': 304, 'Deltona': 71, 'Providence': 552, 'Du

In [188]:
# append the killed dictionary to the temporary dictionary
for key in killed_dict:
    if key in temporary:
        temporary[key] = temporary[key] + [killed_dict[key]]

In [259]:
# append the incident dictionary to the temporary dictionary
for key in incidents_dict:
    if key in temporary:
        temporary[key] = temporary[key] + [incidents_dict[key]]

In [261]:
# make a dictionary of the data in the clean database (which had longitude/latitude)
new_dict = {}
for i in range(len(data)):
    if pd.isna(longitude[i]) == False:
        new_dict[cities[i]] = [latitude[i], longitude[i]]

In [262]:
# append the killed dictionary and incidents dictionary to the new dictionary
for key in new_dict:
    if key in killed_dict:
        new_dict[key] = new_dict[key] + [killed_dict[key]]
    if key in incidents_dict:
        new_dict[key] = new_dict[key] + [incidents_dict[key]]

In [263]:
# merge two dictionaries
for key in temporary:
    if key not in new_dict:
        new_dict[key] = temporary[key]

In [264]:
# make a list of all the cities that miss data
delete_list = []
for key in new_dict:
    if len(new_dict[key]) != 4:
        delete_list.append(key)

In [265]:
# delete those cities
for item in delete_list:
    del(new_dict[item])

In [266]:
df_cities = []
df_long = []
df_lat = []
df_killed = []
df_incidents = []

# make a dataframe of the dictionary
for key in new_dict:
    df_cities.append(key)
    df_long.append(new_dict[key][1])
    df_lat.append(new_dict[key][0])
    df_killed.append(new_dict[key][2])
    df_incidents.append(new_dict[key][3])

d = {'cities': df_cities, 'killed': df_killed, 'incidents': df_incidents, 'latitude': df_lat, 'longitude': df_long}
df = pd.DataFrame(data=d)

In [ ]:
# sort the dataframe for number of killed people and set it in a csv
df = df.sort_values(['killed'], ascending=True)
df.to_csv("bubble_map_killed.csv")

In [272]:
# sort the dataframe for number of incidents and set it in a csv
df = df.sort_values(['incidents'], ascending=True)
df.to_csv("bubble_map_incidents.csv")

In [270]:
import plotly.plotly as py
import numpy as np
import plotly.tools as tls

tls.set_credentials_file(username="ElishaNieuwburg", api_key="61kGTvRZ2BoIejY99Nbo")

In [281]:
df = pd.read_csv('bubble_map_killed.csv')
df.head()

df['text'] = df['cities'] + '<br>Killed ' + (df['killed']).astype(str)

# make sub dataframes for each category
limits = {'0 < 50':(0,6766),'50 < 100':(6766,6835),'100 < 500':(6835,6909),'500 < 1000':(6909,6919),'1000+':(6919,6922)}
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","lightgrey","rgb(255,133,27)"]
cities = []
i = 0

# define every city per category
for key in limits:
    lim = limits[key]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['killed'],
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = key )
    cities.append(city)
    i += 1

layout = dict(
        title = 'Killed per City<br>(Click legend to trace per category)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )

py.iplot( fig, validate=False, filename='killed_per_city' )

                             cities  killed  latitude  longitude
0                  North Fort Myers       1   26.6463   -81.9161
1                           Mattoon       1   39.4806   -88.3711
2                          Hannibal       1   39.7115   -91.3876
3                      Danielsville       1   34.1698   -83.2257
4                  Spokane (county)       1   47.6527  -117.4260
5                        Port Royal       1   32.3862   -80.7029
6                        Saugerties       1   42.0834   -73.9457
7                      Harper Woods       1   42.4480   -82.9157
8                           Statham       1   33.9868   -83.6017
9                          Longmont       1   40.1813  -105.1120
10                         Brooklet       1   32.3296   -81.5228
11                           Victor       1   38.1784   -80.9769
12                           Debary       1   28.8720   -81.3065
13                           Ruskin       1   27.7057   -82.4018
14                    Mou

In [282]:
df = pd.read_csv('bubble_map_incidents.csv')
df.head()

df['text'] = df['cities'] + '<br>Incidents ' + (df['incidents']).astype(str)

# make sub dataframes for each category
limits = {'1 < 100':(0,12222),'100 < 500':(12222,12457),'500 < 1000':(12457,12510),'1000 < 4000':(12510,12542),'10000+':(12542,12543)}
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","lightgrey","rgb(255,133,27)"]
cities = []
scale = 5
i = 0

# define every city per category
for key in limits:
    lim = limits[key]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['incidents']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = key )
    cities.append(city)
    i += 1

layout = dict(
        title = 'Incidents per City<br>(Click legend to trace per category)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )


py.iplot( fig, validate=False, filename='incidents_per_city' )